In [1]:
import intera_interface
import rospy
import json
import tf
from copy import deepcopy
from sawyer import transformations
from os import environ

In [2]:
environ['ROS_MASTER_URI'] = 'http://mingew.local:11311'

In [3]:
environ['ROS_HOSTNAME'] = 'mingew.local'

In [4]:
rospy.init_node("sawyer_example")

In [5]:
effector = "right_hand"
world = "world"

In [6]:
tfl = tf.TransformListener(interpolate=False, cache_time=rospy.Duration(100))
tfb = tf.TransformBroadcaster()

In [7]:
rs = intera_interface.RobotEnable(intera_interface.CHECK_VERSION)

[WARN] [1555447325.301303]: RobotEnable: Failed to retrieve robot version from rosparam: /manifest/robot_software/version/HLR_VERSION_STRING
Verify robot state and connectivity (i.e. ROS_MASTER_URI)


SystemExit: 1

/usr/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [8]:
rs.enable()

[INFO] [1555429213.580470]: Robot Enabled


In [9]:
limb = intera_interface.Limb('right')

In [10]:
hope_lower_left = {u'right_j0': 1.611796875,
                   u'right_j1': -2.75679296875,
                   u'right_j2': -3.0216201171875,
                   u'right_j3': 1.296830078125,
                   u'right_j4': 1.161048828125,
                   u'right_j5': -1.98382421875,
                   u'right_j6': 1.0612880859375}

In [16]:
limb.set_joint_position_speed(0.1)

In [28]:
limb.move_to_joint_positions(hope_lower_left)
# Move to extreme lower left in JOINT space

# Compute sticks in cartesian space

In [7]:
from geometry_msgs.msg import PoseStamped

In [8]:
rospy.logwarn("PLACE PEN AT FIRST STICK")

[WARN] [1555453195.437086]: PLACE PEN AT FIRST STICK


In [15]:
#New attempts

In [194]:
limb.endpoint_pose()["position"]

Point(x=-0.41720399440183237, y=-0.6692288088696071, z=0.2904578769797458)

In [9]:
# REMOVETEST
tfb.sendTransform([-0.41720399440183237, -0.6692288088696071, 0.2904578769797458], [-0.0008742649405844391,  -0.5844689901762121,
  0.004962406664173913,  0.8114004003593491], rospy.Time.now(), "right_hand", "world")

In [26]:
ps = PoseStamped()
ps.header.stamp = rospy.Time(0)
ps.header.frame_id = "world"
ps.pose.position.x = limb.endpoint_pose()["position"].x
ps.pose.position.y = limb.endpoint_pose()["position"].y
ps.pose.position.z = limb.endpoint_pose()["position"].z
ps.pose.orientation.z = 1
stick_T_hand = transformations.pose_to_list(tfl.transformPose("right_hand", ps))


NameError: name 'limb' is not defined

In [11]:
stick_T_hand[0][2] += 0.19

NameError: name 'stick_T_hand' is not defined

In [10]:
# REMOVETEST
stick_T_hand = [[-4.718447854656915e-16, -7.771561172376096e-16, 0.18999999999999911],
                [-0.0008742649405844391,
                 -0.5844689901762121,
                 0.004962406664173913,
                 0.8114004003593491]]

In [11]:
# Stick board is perpendical to the world, multiply by conjuguate
stick_T_hand[1] = tf.transformations.quaternion_conjugate(stick_T_hand[1])

In [12]:
# REMOVETEST
tfb.sendTransform(stick_T_hand[0], stick_T_hand[1], rospy.Time.now(), "sth", "right_hand")

In [13]:
ps = PoseStamped()
ps.header.stamp = rospy.Time(0)
ps.header.frame_id = "right_hand"
ps.pose.position.x = stick_T_hand[0][0]
ps.pose.position.y = stick_T_hand[0][1]
ps.pose.position.z = stick_T_hand[0][2]
ps.pose.orientation.x = stick_T_hand[1][0]
ps.pose.orientation.y = stick_T_hand[1][1]
ps.pose.orientation.z = stick_T_hand[1][2]
ps.pose.orientation.w = stick_T_hand[1][3]
stick_T_world = transformations.pose_to_list(tfl.transformPose("world", ps))

In [14]:
stick_T_world 

[[-0.5974162246140444, -0.6700613865474443, 0.35064806634936585],
 [0.0, 0.0, 0.0, 1.0]]

In [15]:
tfb.sendTransform(stick_T_world[0], stick_T_world[1], rospy.Time.now(), "sticks", "world")

In [16]:
# Starting from lower left (drawing direction)
# 
sticks = {"hope": [], "fear": []}
y_interval = 0.01     # Interval of sticks in y direction
z_interval = 0.02      # Interval of sticks in z direction
group_interval = 0.01  # Interval between each group in y direction
num_groups = 8         # Number of stick groups per line
num_sticks = 4         # Number of sticks per group
height_stick = 0.03    # Length in z direction
num_height = 8         # Number of sticks in z direction (in height)
num_points_cart = 20   # Number of cartesian points for the actual drawing motion

emotion = "fear"
z_stick = stick_T_world[0][2]
col = 0
for h in range(num_height):
    y_stick = stick_T_world[0][1]
    for g in range(num_groups):
        for i in range(num_sticks):
            # Add the end stick
            rotation = stick_T_world[1]
            position = [stick_T_world[0][0], y_stick, z_stick + height_stick]
            end_stick = [position, rotation]
            tfb.sendTransform(end_stick[0], end_stick[1], rospy.Time.now(), str(col), "world")
            motion = [end_stick]

            # Add the cartesian motion
            for p in range(num_points_cart):
                motion_point = deepcopy(end_stick)
                motion_point[0][2] = z_stick + height_stick - height_stick * (p+1) / num_points_cart
                motion.append(motion_point)
                #if i == 0 and g == 0:
                #    tfb.sendTransform(motion_point, [0, 0, 0, 1], rospy.Time.now(), str(col)+'-'+str(p), "world")
            
            sticks[emotion].append(motion)
            y_stick += y_interval
            col += 1
        y_stick += group_interval
    z_stick += z_interval + height_stick
    if len(sticks["fear"]) == num_groups*num_height*num_sticks/2:
        emotion = "hope"
        z_stick += 0.02
print(len(sticks["hope"]), "+", len(sticks["fear"]), "sticks generated")

(128, '+', 128, 'sticks generated')


In [17]:
sticks['fear'][10]

[[[-0.5974162246140444, -0.5500613865474442, 0.3806480663493659],
  [0.0, 0.0, 0.0, 1.0]],
 [[-0.5974162246140444, -0.5500613865474442, 0.3791480663493659],
  [0.0, 0.0, 0.0, 1.0]],
 [[-0.5974162246140444, -0.5500613865474442, 0.3776480663493659],
  [0.0, 0.0, 0.0, 1.0]],
 [[-0.5974162246140444, -0.5500613865474442, 0.3761480663493659],
  [0.0, 0.0, 0.0, 1.0]],
 [[-0.5974162246140444, -0.5500613865474442, 0.37464806634936587],
  [0.0, 0.0, 0.0, 1.0]],
 [[-0.5974162246140444, -0.5500613865474442, 0.37314806634936587],
  [0.0, 0.0, 0.0, 1.0]],
 [[-0.5974162246140444, -0.5500613865474442, 0.37164806634936587],
  [0.0, 0.0, 0.0, 1.0]],
 [[-0.5974162246140444, -0.5500613865474442, 0.37014806634936587],
  [0.0, 0.0, 0.0, 1.0]],
 [[-0.5974162246140444, -0.5500613865474442, 0.36864806634936587],
  [0.0, 0.0, 0.0, 1.0]],
 [[-0.5974162246140444, -0.5500613865474442, 0.36714806634936586],
  [0.0, 0.0, 0.0, 1.0]],
 [[-0.5974162246140444, -0.5500613865474442, 0.36564806634936586],
  [0.0, 0.0, 0.0,

# Cartesian sticks

In [18]:
hand_T_stick = transformations.inverse_transform(stick_T_hand)

In [19]:
hand_T_stick

[[0.18021223021221203, 0.0008325776778371461, -0.06019018936962003],
 array([-0.00087426, -0.58446899,  0.00496241,  0.8114004 ])]

In [20]:
tfb.sendTransform(hand_T_stick[0], hand_T_stick[1], rospy.Time.now(), "hts", "192")

In [31]:
stick_fk = {"hope" : [], "fear": [], "joints": []}
seed = None
for emotion in ["hope", "fear"]:
    for stick_id, stick in enumerate(sticks[emotion]):
        motion = []
        for point_id, point in enumerate(stick):
            hand_T_world = transformations.multiply_transform(point, hand_T_stick)
            tfb.sendTransform(hand_T_world[0], hand_T_world[1], rospy.Time.now(), "eik", world)
            rospy.sleep(0.01)
            # REMOVETEST UNCOMMENT
            #result = limb.ik_request(transformations.list_to_pose(hand_T_world), joint_seed=seed)
            #if result == False:
            #    raise RuntimeError(emotion + ": can't compute stick start " + str(stick_id))
            #else:
            #    motion.append(result.values())
            #    seed = result
        #stick_fk[emotion].append(motion)
        #stick_fk["joints"] = result.keys()
        #rospy.sleep(0.1)

In [186]:
with open("../config/motions.json", "w") as f:
    json.dump(stick_fk, f)

# Reload cartesian sticks

In [11]:
with open("../config/motions.json") as f:
    stick_fk = json.load(f)

In [12]:
from intera_motion_interface import (
    MotionTrajectory,
    MotionWaypoint,
    MotionWaypointOptions
)

In [13]:
traj  = MotionTrajectory()

In [24]:
traj.set_joint_names(stick_fk["joints"])
traj.

In [25]:
for point in stick_fk["hope"][0]:
    waypoint = MotionWaypoint(ma)
    waypoint.set_joint_angles(point)
    traj.append_waypoint(waypoint)

In [27]:
traj.send_trajectory(timeout=20)

result: True
errorId: ''
trajectory_analysis: 
  planned_duration: 4.09254180956
  measured_duration: 4.099870243
  min_angle_command: [0.6436005593905485, -1.8083957825243826, 1.04663495132743, 1.0935538557482565, -3.0225999873378693, -2.549719594979685, 1.5895995374984153]
  max_angle_command: [0.6805687632735085, 1.1754943508222875e-38, 1.0553534042121082, 1.109561937467399, 1.1754943508222875e-38, 1.1754943508222875e-38, 1.6007634731370282]
  peak_speed_command: [0.1516398975064467, 0.10858441551787587, 0.035762209929474084, 0.06566355988529682, 0.07535013896571861, 0.12243219121370731, 0.045793356369000254]
  peak_accel_command: [1.2253422532503047, 0.8774278706755578, 0.28898032520978995, 0.5306013506298057, 0.6088747788757376, 0.9893262889194343, 0.3700380664969747]
  peak_jerk_command: [10.649847802654358, 7.748108409003397, 4.830533143059915, 5.124832771628908, 5.646023849003435, 8.248532415716207, 3.5333986226465397]
  min_time_rate: 1.0
  max_time_rate: 1.0
  max_position_er

In [28]:
traj.clear_waypoints()

In [29]:
w = MotionWaypoint()

In [30]:
traj.set_joint_names(hope_lower_left.keys())
w.set_joint_angles(hope_lower_left.values())

In [31]:
traj.append_waypoint(w)

In [32]:
traj.send_trajectory(timeout=10)

result: True
errorId: ''
trajectory_analysis: 
  planned_duration: 0.92698697522
  measured_duration: 0.929326811
  min_angle_command: [0.6805718709535935, -1.9838242187594355, 1.046635579108908, 1.1095629499611237, -3.0216201132786344, -2.756792968581033, 1.5895996571901037]
  max_angle_command: [1.0612880859375, 1.1754943508222875e-38, 1.1610488281253692, 1.2968300781249995, 1.1754943508222875e-38, 1.1754943508222875e-38, 1.6117968738349906]
  peak_speed_command: [0.7084717308368675, 0.37842880559743924, 0.2468132366852051, 0.40397512051011786, 0.0375130937997645, 0.44669918035683526, 0.04788414741695481]
  peak_accel_command: [2.525521240364423, 1.2517726003458554, 0.9414482266034151, 1.6576309408666876, 0.16054496039042834, 1.91174054050629, 0.20493000634441685]
  peak_jerk_command: [23.152732604838953, 7.768764871376634, 4.677076241363913, 7.543246353955295, 0.698587915175077, 8.318659366822375, 0.8917229512580622]
  min_time_rate: 1.0
  max_time_rate: 1.0
  max_position_error: [0

# Overwrite other poses

In [ ]:
# Pause pose
poses = {"pause": limb.joint_angles()}

with open("../config/poses.json", "w") as f:
    json.dump(poses, f, indent=4)

In [17]:
with open("../config/poses.json") as f:
    poses = json.load(f)

In [ ]:
limb.move_to_joint_positions(poses["pause"])

In [18]:
poses

{u'pause': {u'right_j0': 2.462169921875,
  u'right_j1': -3.327517578125,
  u'right_j2': -2.2095576171875,
  u'right_j3': -1.52369140625,
  u'right_j4': 0.1305107421875,
  u'right_j5': -1.00734375,
  u'right_j6': -0.2455576171875}}

# Display sticks

In [ ]:
with open("../config/sticks_target_ik.json") as f:
    sticks = json.load(f)

In [ ]:
from sensor_msgs.msg import JointState
p = rospy.Publisher("display/joint_states", JointState, queue_size=1)

In [ ]:
type = 'fear'
side = 'end'
for i, pose in enumerate(sticks[type]):
    js = JointState()
    js.header.stamp = rospy.Time.now()
    js.name = pose[side].keys()
    js.position = pose[side].values()
    p.publish(js)
    input = raw_input('Press to display {} pose {} of [q to quit] '.format(side, i, type))
    if input == 'q': break

In [ ]:
sticks

In [ ]:
tf.transformations.